Aggregations refer to any data transformation that produces scalar values from
arrays.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                    'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.640784,0.824034
1,a,two,-0.520735,0.247836
2,b,one,-0.119800,-1.193864
3,b,two,-0.809194,0.798089
4,a,one,1.087455,-0.704983


While quantile is not explicitly implemented for GroupBy, it is a Series method and thus available for use. Internally, GroupBy efficiently slices up the Series, calls piece.quantile(0.9) for each piece, and then assembles those results together into the result object:

In [3]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.765817
b   -0.188739
Name: data1, dtype: float64

To use your own aggregation functions, pass any function that aggregates an array to
the aggregate or agg method:

In [4]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

You may notice that some methods like describe also work, even though they are not
aggregations, strictly speaking:

In [5]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.024688  0.965012 -0.640784 -0.580759 -0.520735  0.283360   
b      2.0 -0.464497  0.487475 -0.809194 -0.636846 -0.464497 -0.292149   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.087455   3.0  0.122296  0.772200 -0.704983 -0.228574  0.247836   
b    -0.119800   2.0 -0.197887  1.408524 -1.193864 -0.695876 -0.197887   

                          
           75%       max  
key1                      
a     0.535935  0.824034  
b     0.300101  0.798089

### Column-Wise and Multiple Function Application

In [7]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [8]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

If you pass a list of functions or function names instead, you get back a DataFrame
with column names taken from the functions:

In [9]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

# Here we passed a list of aggregation functions to agg to evaluate indepedently on the data groups.

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

You don’t need to accept the names that GroupBy gives to the columns; notably,
lambda functions have the name '<lambda>', which makes them hard to identify
(you can see for yourself by looking at a function’s __name__ attribute). Thus, if you
pass a list of (name, function) tuples, the first element of each tuple will be used as
the DataFrame column names (you can think of a list of 2-tuples as an ordered
mapping):

In [10]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame you have more options, as you can specify a list of functions to
apply to all of the columns or different functions per column. To start, suppose we
wanted to compute the same three statistics for the tip_pct and total_bill
columns:

In [12]:
grouped

In [13]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [14]:
result['tip_pct']

NameError: name 'result' is not defined

In [15]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

Now, suppose you wanted to apply potentially different functions to one or more of
the columns. To do this, pass a dict to agg that contains a mapping of column names
to any of the function specifications listed so far:

In [16]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [17]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

A DataFrame will have hierarchical columns only if multiple functions are applied to
at least one column.

### Returning Aggregated Data Without Row Indexes

In all of the examples up until now, the aggregated data comes back with an index,
potentially hierarchical, composed from the unique group key combinations. Since
this isn’t always desirable, you can disable this behavior in most cases by passing
as_index=False to groupby:

In [18]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

TypeError: Could not convert MaleFemaleMaleFemale to numeric

Of course, it’s always possible to obtain the result in this format by calling
reset_index on the result. Using the as_index=False method avoids some unneces‐
sary computations.